# conversion of (large) files and streams

The python SDK supports importing large sets of time series using files and streams in a CSV format.

The SDK converts the file to a local ETL Import file and uploads this file to waylay for an asynchronous import.

This notebook illustrates the various csv import formats that are supported by the SDK (without actually uploading the series)

<img src="ETL File Import.png">

In [1]:
import waylay
waylay.__version__

'v0.3.0+65.gd62232c.dirty'

We use previously created connection profile, and use the `timeseries.etl_tool` for this workflow.

A default temporary file will help cleaning up afterwards.

In [2]:
import tempfile

waylay_client = waylay.WaylayClient.from_profile()
etl_tool = waylay_client.timeseries.etl_tool
etl_tool.temp_dir= tempfile.mkdtemp('etl-import')

### a simple data file
This simple example uses a CSV file where each row is a timestamped event with containing a few measurements.

In [3]:

with open(f'{etl_tool.temp_dir}/hvac_demo.csv','wt') as hvac:
    hvac.write(
"timestamp,temperature,occupancy,humidity,snr,event_id\n"
"2021-02-22T14:35:10+00:00,23,0,2304,0.0001,#xji-98904\n"
"2021-02-22T14:35:20+00:00,21,3,2200,0.0002,#xji-98905\n"
)

Let's check the file exists and that we can read the first two lines:


In [4]:
import csv
with open(f'{etl_tool.temp_dir}/hvac_demo.csv', 'rt') as csv_file:
    csv_reader = csv.reader(csv_file)
    print(next(csv_reader))
    print(next(csv_reader))

['timestamp', 'temperature', 'occupancy', 'humidity', 'snr', 'event_id']
['2021-02-22T14:35:10+00:00', '23', '0', '2304', '0.0001', '#xji-98904']


### preparing the import
Before uploading it into waylay, `etl_tool.prepare_import()` will convert your input source into a local CSV file that
has the right format.

In this case, we provide arguments
* `'hvac_demo.csv'`, our csv file,
* an (optional) `name` for the import action itself (to track its progress)
* the `resource` setting to indicate the _resource id_ to use for the upload (as it is not in the data set)
* the (optional) `metrics` setting to filter the measurements we want to include in the data set.

The call returns a data object that keeps track of the import file and settings, and is the main argument for the other `etl_tool` utility functions.

In [5]:
MODEL_NAME = 'hvac_0001'
hvac_import = etl_tool.prepare_import(
    f'{etl_tool.temp_dir}/hvac_demo.csv',
    name=MODEL_NAME,
    resource='r_489784',
    metrics=['temperature', 'humidity', 'occupancy']
)

1.00csv_files [00:00, 171csv_files/s]
 50%|█████     | 3.00/6.00 [00:00<00:00, 1.63krows/s]


In [6]:
hvac_import

WaylayETLSeriesImport(import_file=ETLFile(directory='/var/folders/07/zn347xhn33z8m79l8xtz1hn80000gp/T/tmpgj0x_gi_etl-import', prefix='hvac_0001'), settings=SeriesSettings(metrics=[temperature, humidity, occupancy], metric_column=None, metric=None, resources=None, resource_column=None, resource='r_489784', value_column=None, timestamp_column='timestamp', timestamp_offset=None, timestamp_first=None, timestamp_last=None, timestamp_interval=None, timestamp_constructor=<function TimestampFormat.parser.<locals>._parse_with_tz at 0x7f962975ddc0>, timestamp_timezone=None, name=None, timestamp_from=None, timestamp_until=None, timestamp_formatter=None, timestamp_format=None, write_csv_header=True, per_resource=False, per_metric=False), storage_bucket='etl-import', stats=SeriesStatistics(inputs=defaultdict(<function SeriesStatistics.<lambda>.<locals>.<lambda> at 0x7f962c418d30>, {'hvac_demo': 1}), row_count=0))

In [7]:
hvac_import.settings.resource

'r_489784'

In [8]:
hvac_import.import_file.path

PosixPath('/var/folders/07/zn347xhn33z8m79l8xtz1hn80000gp/T/tmpgj0x_gi_etl-import/hvac_0001-timeseries.csv.gz')

You can use `etl_tool.read_import_as_dataframe` to validate the content of the *etl import file* etl_tool 

In [11]:
etl_tool.read_import_as_dataframe(hvac_import)

resource                     r_489784                   
metric                    temperature occupancy humidity
timestamp                                               
2021-02-22 14:35:10+00:00        23.0       0.0   2304.0
2021-02-22 14:35:20+00:00        21.0       3.0   2200.0

### initiating the import

The function `etl_tool.initiate_import` will upload the *etl import file*, initiating the import process

In [12]:
hvac_import = etl_tool.initiate_import(hvac_import)

### checking the import process
Use `etl_tool.check_import` to follow up on the import process. The returned object has a `to_html` method that you can use to visualize the status in this notebook:

In [14]:
from IPython.core.display import HTML

HTML(
    etl_tool.check_import(hvac_import).to_html()
)

You can also list all previous import jobs.

The results are kept in the `etl-import` storage bucket, in folders according to the job status.

In [15]:
for import_job in etl_tool.list_import():
    display(HTML(import_job.to_html()))

Once the import process is sucessfull, you can explore and use the data.

### Provisioning a _waylay resource_ or _waylay query_ for the dataset
The `etl_tool` gives you two provisioning utilities that let you create waylay entities related to your dataset:
    
* `etl_tool.update_query` will create a **waylay query** for the dataset. This will allow you to refer and query this dataset using a query name. 

* `etl_tool.update_resources` will create a **waylay resource** for each resource referenced in your dataset. 
   You can also use `etl_tool.list_import_resources` to only render the resource definitions, without actually
   creating them on the platform.


#### provisioning a _waylay query_

In [16]:
etl_tool.update_query(hvac_import)

{'data': [{'metric': 'temperature', 'resource': 'r_489784'},
  {'metric': 'occupancy', 'resource': 'r_489784'},
  {'metric': 'humidity', 'resource': 'r_489784'}],
 'from': '2021-02-22T14:35:10Z'}

Once created, we can used the named query in the console of using the `analytics.query` api.

In [17]:
waylay_client.analytics.query.get(MODEL_NAME)

{'data': [{'metric': 'temperature', 'resource': 'r_489784'},
  {'metric': 'occupancy', 'resource': 'r_489784'},
  {'metric': 'humidity', 'resource': 'r_489784'}],
 'from': '2021-02-22T14:35:10Z'}

In [18]:
waylay_client.analytics.query.data(MODEL_NAME)


resource                     r_489784                   
metric                    temperature occupancy humidity
timestamp                                               
2021-02-22 14:35:10+00:00          23         0     2304
2021-02-22 14:35:20+00:00          21         3     2200

In [19]:
waylay_client.analytics.query.data(MODEL_NAME, params={
    'from': '2021-02-22T14:00:00Z',
    'freq':'PT1H', 
    'aggregation':'mean',
    'periods':2
})

resource                     r_489784                   
metric                    temperature occupancy humidity
aggregation                      mean      mean     mean
timestamp                                               
2021-02-22 14:00:00+00:00        22.0       1.5   2252.0
2021-02-22 15:00:00+00:00         NaN       NaN      NaN

#### provisioning a _waylay resource_

In [20]:
etl_tool.update_resources(hvac_import)

[r_489784]

You can validate on the console, or using the `waylay.api.resource` api that the resource has been created:

In [21]:
waylay_client.api.resource.get('r_489784')

{'id': 'r_489784',
 'name': 'r_489784',
 'metrics': [{'name': 'temperature'},
  {'name': 'occupancy'},
  {'name': 'humidity'}]}

When preparing an import you can specify additional metadata on the resources and metrics in the dataset, which will be taken into account when creating the _waylay resource_. 
This also supports the renaming of metrics and resources as specified in the data set (`Metric.key`, `Resource.key`) to the names used in the waylay upload itself (`Metric.name`. `Resource.id`).

In [19]:
from waylay.service.timeseries import Resource, Metric
hvac_demo_with_metadata = etl_tool.prepare_import(
    f'{etl_tool.temp_dir}/hvac_demo.csv',
    name='hvac_0006',
    resource='r_489786',
    resources=[
        Resource(
            'r_489786', 
            name='Home Office 489786', 
            description='Example Resource for the Waylay SDK etl_tool demo'
        )
    ],
    metrics=[
        Metric(name="temp", key='temperature', value_type="float", metric_type="gauge", unit="°C", description='Home office temperature.'), 
        Metric(name="humi", key="humidity", value_type= "float",  metric_type="gauge",  unit="%", description= "Relative Humidity at my desk."), 
        Metric(name="occu", key="occupancy", value_type="integer",  metric_type="gauge",  unit="items", description="Number of cups on my desk.")
  
    ]
)

100%|██████████| 3.00/3.00 [00:00<00:00, 3.69kseries/s]


In [20]:
etl_tool.update_resources(hvac_demo_with_metadata)

[Resource(id='r_489786', key=None, description='Example Resource for the Waylay SDK etl_tool demo', name='Home Office 489786', metrics=[Metric(name='temp', key='temperature', value_parser=None, description='Home office temperature.', value_type='float', metric_type='gauge', unit='°C'), Metric(name='humi', key='humidity', value_parser=None, description='Relative Humidity at my desk.', value_type='float', metric_type='gauge', unit='%'), Metric(name='occu', key='occupancy', value_parser=None, description='Number of cups on my desk.', value_type='integer', metric_type='gauge', unit='items')])]

In [21]:
waylay_client.api.resource.get('r_489786')

{'id': 'r_489786',
 'name': 'Home Office 489786',
 'metrics': [{'name': 'temp',
   'valueType': 'float',
   'metricType': 'gauge',
   'unit': '°C',
   'description': 'Home office temperature.'},
  {'name': 'humi',
   'valueType': 'float',
   'metricType': 'gauge',
   'unit': '%',
   'description': 'Relative Humidity at my desk.'},
  {'name': 'occu',
   'valueType': 'integer',
   'metricType': 'gauge',
   'unit': 'items',
   'description': 'Number of cups on my desk.'}],
 'description': 'Example Resource for the Waylay SDK etl_tool demo'}

In [22]:
waylay_client.data.events.remove(MODEL_NAME)

{'message': 'Deleted messages, series and all metrics for hvac_0001'}